In [1]:
import googleapiclient.discovery
import tensorflow as tf
import pandas as pd
import json

In [2]:
PROJECT = 'mlops-dev-env'
BUCKET =  'mlops-dev-workspace'
GCS_DATA_LOCATION = 'gs://workshop-datasets/covertype/data_validation/evaluation/dataset.csv'
REGION = 'us-central1'
LOCAL_WORKSPACE = '/home/jarekk/workspace'
LOCAL_DATA_DIR = os.path.join(LOCAL_WORKSPACE, 'data')
CSV_DATA_FILE = '{}/data.csv'.format(LOCAL_DATA_DIR)
JSON_DATA_FILE = '{}/json_instances'.format(LOCAL_DATA_DIR)
BQ_DATASET_NAME = 'data_validation'
BQ_TABLE_NAME = 'covertype_classifier_logs_tf'
MODEL_NAME = 'covertype_classifier_tf'
VERSION_NAME = 'v2'
TRAINING_DIR = os.path.join(LOCAL_WORKSPACE, 'training')
MODEL_DIR = os.path.join(TRAINING_DIR, 'exported_model')

In [14]:
if tf.io.gfile.exists(LOCAL_WORKSPACE):
  print("Removing previous workspace artifacts...")
  tf.io.gfile.rmtree(LOCAL_WORKSPACE)

print("Creating a new workspace...")
tf.io.gfile.makedirs(LOCAL_WORKSPACE)
tf.io.gfile.makedirs(LOCAL_DATA_DIR)

Removing previous workspace artifacts...
Creating a new workspace...


In [15]:
!gsutil ls {GCS_DATA_LOCATION}

gs://workshop-datasets/covertype/data_validation/evaluation/dataset.csv


In [16]:
!gsutil cp {GCS_DATA_LOCATION} {CSV_DATA_FILE}


Copying gs://workshop-datasets/covertype/data_validation/evaluation/dataset.csv...

Operation completed over 1 objects/3.9 MiB.                                      


In [9]:
!head {CSV_DATA_FILE}

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
2814,55,17,30,8,2483,228,201,100,2727,Rawah,7202,1
2903,149,4,108,7,5148,225,239,149,4713,Rawah,6101,1
3338,236,10,300,27,1008,202,249,184,1620,Commanche,7756,0
3305,153,20,60,9,2353,240,237,118,324,Commanche,8703,6
2788,107,7,0,0,2533,233,232,131,3896,Rawah,7202,1
2609,101,35,170,68,1050,252,172,17,624,Commanche,4758,2
3175,38,10,210,38,1537,220,217,131,95,Rawah,7201,0
2606,253,7,417,135,547,204,245,179,503,Commanche,2703,2
2945,90,3,240,78,600,224,234,146,1803,Commanche,7756,0


In [11]:
df = pd.read_csv(CSV_DATA_FILE)
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2814,55,17,30,8,2483,228,201,100,2727,Rawah,7202,1
1,2903,149,4,108,7,5148,225,239,149,4713,Rawah,6101,1
2,3338,236,10,300,27,1008,202,249,184,1620,Commanche,7756,0
3,3305,153,20,60,9,2353,240,237,118,324,Commanche,8703,6
4,2788,107,7,0,0,2533,233,232,131,3896,Rawah,7202,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,3294,336,14,268,-37,1460,187,220,171,1945,Neota,7202,0
74996,3372,74,31,408,138,1860,239,167,35,2063,Rawah,8776,0
74997,3295,225,16,319,1,1739,196,253,190,3997,Neota,7756,0
74998,3125,84,10,283,43,1980,234,224,121,3264,Commanche,7700,0


In [14]:
with open(JSON_DATA_FILE, 'w') as f:
    for row in df.iterrows():
        feature_dict = {key: [value] for key, value in row[1].to_dict().items()}
        feature_dict['Soil_Type'] = [str(feature_dict['Soil_Type'][0])]
        json_object = json.dumps(feature_dict)
        f.write(json_object)
        f.write('\n')

In [15]:
!head {JSON_DATA_FILE}

{"Elevation": [2814], "Aspect": [55], "Slope": [17], "Horizontal_Distance_To_Hydrology": [30], "Vertical_Distance_To_Hydrology": [8], "Horizontal_Distance_To_Roadways": [2483], "Hillshade_9am": [228], "Hillshade_Noon": [201], "Hillshade_3pm": [100], "Horizontal_Distance_To_Fire_Points": [2727], "Wilderness_Area": ["Rawah"], "Soil_Type": ["7202"], "Cover_Type": [1]}
{"Elevation": [2903], "Aspect": [149], "Slope": [4], "Horizontal_Distance_To_Hydrology": [108], "Vertical_Distance_To_Hydrology": [7], "Horizontal_Distance_To_Roadways": [5148], "Hillshade_9am": [225], "Hillshade_Noon": [239], "Hillshade_3pm": [149], "Horizontal_Distance_To_Fire_Points": [4713], "Wilderness_Area": ["Rawah"], "Soil_Type": ["6101"], "Cover_Type": [1]}
{"Elevation": [3338], "Aspect": [236], "Slope": [10], "Horizontal_Distance_To_Hydrology": [300], "Vertical_Distance_To_Hydrology": [27], "Horizontal_Distance_To_Roadways": [1008], "Hillshade_9am": [202], "Hillshade_Noon": [249], "Hillshade_3pm": [184], "Horizonta

In [34]:
!cat {JSON_DATA_FILE} | wc -l

75000


In [35]:
len(df)

75000

In [2]:
def my_iterator(l1):
    for item in l1:
        yield item

In [18]:
l1 = [1, 2, 3]

In [19]:
iter1 = my_iterator(l1)

In [20]:
type(iter1)

generator

In [22]:
next(iter1)

2

In [16]:
m.next()

AttributeError: 'generator' object has no attribute 'next'

In [2]:
tfdv.__version__

'0.22.0'

In [1]:
file = '/home/jarekk/ttt/test.tfrecords'

In [2]:
!ls {file}

/home/jarekk/ttt/test.tfrecords


In [12]:
import tensorflow_data_validation as tfdv
import pandas as pd
from tensorflow_data_validation.utils import slicing_util


In [10]:
file = '/home/jarekk/workspace/data/eval.csv'

In [13]:
df = pd.read_csv(file)
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2814,55,17,30,8,2483,228,201,100,2727,Rawah,7202,1
1,2903,149,4,108,7,5148,225,239,149,4713,Rawah,6101,1
2,3338,236,10,300,27,1008,202,249,184,1620,Commanche,7756,0
3,3305,153,20,60,9,2353,240,237,118,324,Commanche,8703,6
4,2788,107,7,0,0,2533,233,232,131,3896,Rawah,7202,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,3294,336,14,268,-37,1460,187,220,171,1945,Neota,7202,0
74996,3372,74,31,408,138,1860,239,167,35,2063,Rawah,8776,0
74997,3295,225,16,319,1,1739,196,253,190,3997,Neota,7756,0
74998,3125,84,10,283,43,1980,234,224,121,3264,Commanche,7700,0


In [14]:
slice_fn = slicing_util.get_feature_value_slicer(features={'Wilderness_Area': None})

stats_options = tfdv.StatsOptions(
    slice_functions=[slice_fn])

#stats = tfdv.generate_statistics_from_csv(
#    data_location=file,
#    stats_options=stats_options)

stats = tfdv.generate_statistics_from_dataframe(
    dataframe=df,
    stats_options=stats_options)


In [15]:
tfdv.visualize_statistics(stats)